<a href="https://colab.research.google.com/github/shivanic02/SpaceHACK-2025/blob/main/SpaceHACK_Track_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaurang

In [ ]:

import ee
from google.oauth2 import service_account

# Path to your downloaded JSON key file
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'

# Service account email
SERVICE_ACCOUNT_EMAIL = 'spacehack@spacehack2025-454422.iam.gserviceaccount.com'

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)

# Initialize Earth Engine with credentials
ee.Initialize(credentials)


In [ ]:
import ee
import folium
from google.oauth2 import service_account

# Authenticate and initialize Earth Engine
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials)

# Define Phoenix point and 15 km buffer
phoenix_point = ee.Geometry.Point([-112.0740, 33.4484])
phoenix_buffer = phoenix_point.buffer(15000)  # 15 km buffer

# --- Satellite Datasets and Masks ---

# 1. NDVI from Sentinel-2
sentinel2 = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(phoenix_buffer) \
    .filterDate('2020-01-01', '2020-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
ndvi = sentinel2.mean().normalizedDifference(['B8', 'B4']).rename('NDVI').clip(phoenix_buffer)
low_vegetation = ndvi.lt(0.2)

# 2. Soil Moisture from SMAP
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007") \
    .filterBounds(phoenix_buffer) \
    .filterDate('2020-01-01', '2020-12-31')
soil_moisture = smap.mean().select('sm_surface').clip(phoenix_buffer)
suitable_soil = soil_moisture.gt(0.2)

# 3. LULC - Barren Land from NLCD
lulc = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD") \
    .sort('system:time_start', False).first() \
    .select('landcover') \
    .clip(phoenix_buffer)
barren_land = lulc.eq(31)

# 4. Impervious Surface from NLCD
impervious = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD") \
    .sort('system:time_start', False).first() \
    .select('impervious') \
    .clip(phoenix_buffer)
low_impervious = impervious.lt(20)

# 5. Land Surface Temperature from MODIS
modis_lst = ee.ImageCollection("MODIS/006/MOD11A2") \
    .filterBounds(phoenix_buffer) \
    .filterDate('2020-01-01', '2020-12-31') \
    .select('LST_Day_1km') \
    .mean() \
    .multiply(0.02).subtract(273.15).clip(phoenix_buffer)
heat_stress = modis_lst.gt(35)

# 6. Population Density (GPW UNWPP Adjusted)
population = ee.ImageCollection("CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density") \
    .filterDate('2020-01-01', '2020-12-31') \
    .mean() \
    .clip(phoenix_buffer)
high_density = population.gt(1000)

# --- Combine All Conditions for Urban Agriculture Suitability ---
suitable_sites = barren_land.multiply(low_vegetation).multiply(suitable_soil)
priority_zones = suitable_sites.multiply(low_impervious).multiply(heat_stress).multiply(high_density)

# --- Visualization Map ---

# Add Earth Engine layer method to folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# Create base map
my_map = folium.Map(location=[33.4484, -112.0740], zoom_start=12)

# NDVI – visualize low vegetation in brown
ndvi_vis = {'min': 1, 'max': 1, 'palette': ['#a0522d']}
my_map.add_ee_layer(low_vegetation.selfMask(), ndvi_vis, 'Low Vegetation (NDVI < 0.2)')

# Barren land – visualize in black
barren_vis = {'min': 1, 'max': 1, 'palette': ['black']}
my_map.add_ee_layer(barren_land.selfMask(), barren_vis, 'Barren Land')

# Suitable soil – visualize in teal
soil_vis = {'min': 1, 'max': 1, 'palette': ['#40e0d0']}
my_map.add_ee_layer(suitable_soil.selfMask(), soil_vis, 'Suitable Soil')

# Population – visualize high density
#pop_vis = {'min': 1, 'max': 1, 'palette': ['lightblue']}
#my_map.add_ee_layer(high_density.selfMask(), pop_vis, 'High Population Density')

# Final priority zones – visualize in green
priority_vis = {'min': 1, 'max': 1, 'palette': ['green']}
my_map.add_ee_layer(priority_zones.selfMask(), priority_vis, 'Urban Agriculture Priority Zones')

# Add 15 km buffer boundary
folium.GeoJson(
    phoenix_buffer.getInfo(),
    name='15 km Buffer',
    style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
).add_to(my_map)

# Layer control
folium.LayerControl().add_to(my_map)

# Save map
my_map.save("phoenix_food_security_map.html")
my_map


In [ ]:
import ee
import folium
from google.oauth2 import service_account

# Authenticate and initialize Earth Engine
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials)

# Define Phoenix point and 15 km buffer
phoenix_point = ee.Geometry.Point([-112.0740, 33.4484])
phoenix_buffer = phoenix_point.buffer(15000)  # 15 km buffer

# --- Satellite Datasets and Masks ---

# 1. NDVI from Sentinel-2
sentinel2 = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(phoenix_buffer) \
    .filterDate('2020-01-01', '2020-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
ndvi = sentinel2.mean().normalizedDifference(['B8', 'B4']).rename('NDVI').clip(phoenix_buffer)
low_vegetation = ndvi.lt(0.2)

# 2. Soil Moisture from SMAP
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007") \
    .filterBounds(phoenix_buffer) \
    .filterDate('2020-01-01', '2020-12-31')
soil_moisture = smap.mean().select('sm_surface').clip(phoenix_buffer)
suitable_soil = soil_moisture.gt(0.2)

# 3. LULC - Barren Land from NLCD
lulc = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD") \
    .sort('system:time_start', False).first() \
    .select('landcover') \
    .clip(phoenix_buffer)
barren_land = lulc.eq(31)

# 4. Impervious Surface from NLCD
impervious = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD") \
    .sort('system:time_start', False).first() \
    .select('impervious') \
    .clip(phoenix_buffer)
low_impervious = impervious.lt(20)

# 5. Land Surface Temperature from MODIS
modis_lst = ee.ImageCollection("MODIS/006/MOD11A2") \
    .filterBounds(phoenix_buffer) \
    .filterDate('2020-01-01', '2020-12-31') \
    .select('LST_Day_1km') \
    .mean() \
    .multiply(0.02).subtract(273.15).clip(phoenix_buffer)
heat_stress = modis_lst.gt(35)

# 6. Population Density (GPW UNWPP Adjusted)
population = ee.ImageCollection("CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density") \
    .filterDate('2020-01-01', '2020-12-31') \
    .mean() \
    .clip(phoenix_buffer)
high_density = population.gt(1000)

# --- Combine All Conditions for Urban Agriculture Suitability ---
suitable_sites = barren_land.multiply(low_vegetation).multiply(suitable_soil)
priority_zones = suitable_sites.multiply(low_impervious).multiply(heat_stress).multiply(high_density)

# --- Visualization Map ---

# Add Earth Engine layer method to folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# Create base map
my_map = folium.Map(location=[33.4484, -112.0740], zoom_start=12)

# NDVI – visualize low vegetation in brown
ndvi_vis = {'min': 1, 'max': 1, 'palette': ['red'], 'opacity': 0.5}
my_map.add_ee_layer(low_vegetation.selfMask(), ndvi_vis, 'Low Vegetation (NDVI < 0.2)')

# Barren land – visualize in black
barren_vis = {'min': 1, 'max': 1, 'palette': ['black']}
my_map.add_ee_layer(barren_land.selfMask(), barren_vis, 'Barren Land')

# Suitable soil – visualize in teal
soil_vis = {'min': 1, 'max': 1, 'palette': ['grey']}
my_map.add_ee_layer(suitable_soil.selfMask(), soil_vis, 'Suitable Soil')

# Population – visualize high density
pop_vis = {'min': 1, 'max': 1, 'palette': ['purple']}
my_map.add_ee_layer(high_density.selfMask(), pop_vis, 'High Population Density')

# Final priority zones – visualize in green
priority_vis = {'min': 1, 'max': 1, 'palette': ['darkgreen']}
my_map.add_ee_layer(priority_zones.selfMask(), priority_vis, 'Urban Agriculture Priority Zones')

# Add 15 km buffer boundary
folium.GeoJson(
    phoenix_buffer.getInfo(),
    name='15 km Buffer',
    style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
).add_to(my_map)

# Layer control
folium.LayerControl().add_to(my_map)

# Save map
my_map.save("phoenix_food_security_map.html")
my_map


In [ ]:
collections = ee.data.getList({'id': 'LANDSAT'})
for c in collections:
    print(c['id'])



projects/earthengine-public/assets/LANDSAT/COMPOSITES
projects/earthengine-public/assets/LANDSAT/GLS1975
projects/earthengine-public/assets/LANDSAT/GLS1975_MOSAIC
projects/earthengine-public/assets/LANDSAT/GLS1975_TOA_MOSAIC
projects/earthengine-public/assets/LANDSAT/GLS2005
projects/earthengine-public/assets/LANDSAT/GLS2005_L5
projects/earthengine-public/assets/LANDSAT/GLS2005_L7
projects/earthengine-public/assets/LANDSAT/GLS2005_MAPPED
projects/earthengine-public/assets/LANDSAT/L4
projects/earthengine-public/assets/LANDSAT/L4_L1T
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_BAI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_EVI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NBRT
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NDSI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NDVI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NDWI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_RAW
projects/earthengine-public/assets/LAN

In [ ]:
lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9")

In [ ]:
# Example: land cover for 2021
lcms_2021 = lcms.filter(ee.Filter.eq('year', 2021)) \
                .filterBounds(ee.Geometry.Rectangle([-114.8, 31.3, -109, 37]))  # Arizona BBox


In [ ]:
land_cover = lcms_2021.select('Land_Cover').mosaic()
land_use = lcms_2021.select('Land_Use').mosaic()
phoenix = ee.Geometry.Point([-112.0740, 33.4484]).buffer(30000)  # 30 km buffer
tucson = ee.Geometry.Point([-110.9747, 32.2226]).buffer(30000)
# Assume class 1 = Developed Open Space, class 6 = Shrubland
suitable_land = land_use.eq(1).Or(land_use.eq(6))


In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
             .filterDate('2021-01-01', '2021-12-31') \
             .filterBounds(phoenix) \
             .median()

# Compute NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
green_mask = ndvi.gte(0.3)  # Threshold for vegetative health
intervention_zones_phoenix = suitable_land.And(green_mask).clip(phoenix)
intervention_zones_tucson = suitable_land.And(green_mask).clip(tucson)


In [ ]:
import ee
import folium
from google.oauth2 import service_account

# Authenticate and initialize Earth Engine
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials)

# Define Phoenix location (30 km buffer)
phoenix = ee.Geometry.Point([-112.0740, 33.4484]).buffer(30000)

# Load LCMS 2021 Land Use data
lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9")
lcms_2021 = lcms.filter(ee.Filter.eq('year', 2021)).select('Land_Use').mosaic()

# Choose suitable land use classes (example: 1 = Open Space, 6 = Shrubland)
suitable_land = lcms_2021.eq(6).Or(lcms_2021.eq(1)).Or(lcms_2021.eq(3))

# Load Landsat 8 imagery for NDVI
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2021-01-01', '2021-12-31') \
    .filterBounds(phoenix) \
    .median()

ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Filter areas with NDVI >= 0.3 (vegetation)
green_mask = ndvi.gte(0.3)

# Final mask: land must be suitable AND green
green_zones = suitable_land.And(green_mask).selfMask().clip(phoenix)

# Display map using folium
map_phoenix = folium.Map(location=[33.4484, -112.0740], zoom_start=11)

map_id = green_zones.getMapId({
    'min': 1,
    'max': 1,
    'palette': ['green']
})

folium.TileLayer(
    tiles=map_id['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='Green Zones (Phoenix)',
    overlay=True,
    control=True
).add_to(map_phoenix)

map_phoenix.add_child(folium.LayerControl())
map_phoenix
map_phoenix.save("bitch.html")

In [ ]:
import ee
import geemap
import geopandas as gpd

ee.Initialize(credentials)

# Create map
Map = geemap.Map(center=[34, -111.5], zoom=6)

# Define full Arizona boundary
arizona = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Arizona'))

# Load Landsat 8 and compute NDVI
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2022-01-01', '2022-12-31') \
    .filterBounds(arizona.geometry()) \
    .median()

ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Mask NDVI >= 0.3
green_zones = ndvi.gte(0.3).selfMask().clip(arizona)

# Load CDL and select 'cropland' band (adjust if needed)
cdl_2022 = ee.ImageCollection('USDA/NASS/CDL') \
    .filterDate('2022-01-01', '2022-12-31') \
    .first() \
    .select('cropland') \
    .clip(arizona)

# Add layers to map
Map.addLayer(green_zones, {'min': 0, 'max': 1, 'palette': ['green'], 'opacity': 0.5}, 'Green Zones NDVI ≥ 0.3')
Map.addLayer(cdl_2022, {'min': 0, 'max': 255, 'palette': ['000000', 'FFFF00', '00FF00', 'FF0000']}, 'USDA Cropland 2022')
Map.addLayer(arizona.style(color='0000FF', fillColor='00000000', width=2), {}, 'Arizona Border')
Map.addLayerControl()
Map
# Save the map as an HTML file
#Map.to_html('arizona_map.html')

Map(center=[34, -111.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [ ]:
import ee
import folium
from google.oauth2 import service_account

# Authenticate and initialize Earth Engine
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials)

# Define Phoenix location (30 km buffer)
phoenix = ee.Geometry.Point([-112.0740, 33.4484]).buffer(30000)

# Load LCMS 2021 Land Use data
lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9")
lcms_2021 = lcms.filter(ee.Filter.eq('year', 2021)).select('Land_Use').mosaic()

# Choose suitable land use classes (example: 1 = Open Space, 6 = Shrubland, 3 = another class)
suitable_land = lcms_2021.eq(6).Or(lcms_2021.eq(1)).Or(lcms_2021.eq(3))

# Load Landsat 8 imagery for NDVI
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2021-01-01', '2021-12-31') \
    .filterBounds(phoenix) \
    .median()

ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Filter areas with NDVI >= 0.3 (vegetation)
green_mask = ndvi.gte(0.3)

# Final mask: land must be suitable AND green
green_zones = suitable_land.And(green_mask).selfMask().clip(phoenix)

# Create folium map centered on Phoenix
map_phoenix = folium.Map(location=[33.4484, -112.0740], zoom_start=11)

# Add the green zones layer from Earth Engine
map_id = green_zones.getMapId({
    'min': 1,
    'max': 1,
    'palette': ['green']
})

folium.TileLayer(
    tiles=map_id['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='Green Zones (Phoenix)',
    overlay=True,
    control=True
).add_to(map_phoenix)

# --- Add Boundary for Scottsdale including Salt River Project ---

# Option 2: Manually define an approximate Scottsdale boundary
scottsdale_coords = [
    [-111.98, 33.45],
    [-111.98, 33.70],
    [-111.80, 33.70],
    [-111.80, 33.45],
    [-111.98, 33.45]
]
scottsdale_geom = ee.Geometry.Polygon([scottsdale_coords])

# Define an approximate polygon for the Salt River Project area
salt_river_project = ee.Geometry.Polygon([
    [
        [-112.15, 33.60],
        [-112.15, 33.80],
        [-111.90, 33.80],
        [-111.90, 33.60],
        [-112.15, 33.60]
    ]
])

# Combine the two geometries (union of Scottsdale boundary and SRP area)
combined_boundary = scottsdale_geom.union(salt_river_project)

# Get GeoJSON representation of the combined boundary
boundary_geojson = combined_boundary.getInfo()

# Add the combined boundary as a GeoJSON layer with a blue outline
folium.GeoJson(
    boundary_geojson,
    name="Scottsdale + SRP Boundary",
    style_function=lambda feature: {
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(map_phoenix)

# Add layer control to toggle overlays
map_phoenix.add_child(folium.LayerControl())


# Display the map
map_phoenix
map_phoenix.save("map_phoenix.html")


In [ ]:
import ee
import folium
from google.oauth2 import service_account

# Authenticate and initialize Earth Engine
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials)

# Define Phoenix location (30 km buffer)
phoenix = ee.Geometry.Point([-112.0740, 33.4484]).buffer(30000)

# Load LCMS 2021 Land Use data
lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9")
lcms_2021 = lcms.filter(ee.Filter.eq('year', 2021)).select('Land_Use').mosaic()

# Choose suitable land use classes (example: 1 = Open Space, 6 = Shrubland, 3 = another class)
suitable_land = lcms_2021.eq(6).Or(lcms_2021.eq(1)).Or(lcms_2021.eq(3))

# Load Landsat 8 imagery for NDVI
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2021-01-01', '2021-12-31') \
    .filterBounds(phoenix) \
    .median()

ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Filter areas with NDVI >= 0.3 (vegetation)
green_mask = ndvi.gte(0.3)

# Final mask: land must be suitable AND green
green_zones = suitable_land.And(green_mask).selfMask().clip(phoenix)

# Create folium map centered on Phoenix
map_phoenix = folium.Map(location=[33.4484, -112.0740], zoom_start=11)

# Add the green zones layer from Earth Engine
map_id = green_zones.getMapId({
    'min': 1,
    'max': 1,
    'palette': ['green']
})

folium.TileLayer(
    tiles=map_id['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='Green Zones (Phoenix)',
    overlay=True,
    control=True
).add_to(map_phoenix)

# --- Add a boundary of radius 10km with center at (33.465453, -111.865735) ---

new_boundary = ee.Geometry.Point([-111.865735, 33.465453]).buffer(20000)
new_boundary_geojson = new_boundary.getInfo()

folium.GeoJson(
    new_boundary_geojson,
    name="10km Boundary",
    style_function=lambda feature: {
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(map_phoenix)

# Add layer control to toggle overlays
map_phoenix.add_child(folium.LayerControl())
map_phoenix

# Save the map
#map_phoenix.save("map_phoenix.html")


food access

In [ ]:
# Install if needed
!pip install geemap geopandas folium

import ee
import geemap
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# --- Authenticate Earth Engine ---
from google.oauth2 import service_account
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials)

# --- Load Arizona boundary ---
arizona = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Arizona'))

# --- Create Map ---
Map = geemap.Map(center=[34, -111.5], zoom=6)

# --- NDVI Green Zones ---
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2022-01-01', '2022-12-31') \
    .filterBounds(arizona.geometry()) \
    .median()

ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
green_zones = ndvi.gte(0.3).selfMask().clip(arizona)

# --- USDA Cropland Data Layer ---
cdl_2022 = ee.ImageCollection('USDA/NASS/CDL') \
    .filterDate('2022-01-01', '2022-12-31') \
    .first() \
    .select('cropland') \
    .clip(arizona)

# --- Load & Filter Food Access Data ---
df = pd.read_csv('/content/Food Access Research Atlas.csv', encoding='latin1')

# Filter for Arizona, high-income + low-access tracts
df_az = df[df['State'] == 'AZ']
filtered = df_az[
    (df_az['LowIncomeTracts'] == 0) &
    ((df_az['LA1and10'] == 1) | (df_az['LATracts1'] == 1) | (df_az['LATracts10'] == 1))
].dropna(subset=['Latitude', 'Longitude'])

# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(filtered['Longitude'], filtered['Latitude'])]
gdf_az_hi_lo = gpd.GeoDataFrame(filtered, geometry=geometry, crs="EPSG:4326")

# --- Add Layers to Map ---
Map.addLayer(green_zones, {'min': 0, 'max': 1, 'palette': ['green'], 'opacity': 0.5}, 'Green Zones (NDVI ≥ 0.3)')
Map.addLayer(cdl_2022, {'min': 0, 'max': 255, 'palette': ['000000', 'FFFF00', '00FF00', 'FF0000']}, 'Cropland 2022')
Map.add_gdf(gdf_az_hi_lo, layer_name='High-Income Low-Access Tracts')
Map.addLayer(arizona.style(color='blue', fillColor='00000000', width=2), {}, 'Arizona Boundary')

Map.addLayerControl()
Map


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00


KeyError: ['Latitude', 'Longitude']

In [ ]:
# --- Load & Filter Food Access Data ---
df = pd.read_csv('/content/Food Access Research Atlas.csv', encoding='latin1')
print(df.columns[df.columns.str.contains('lat|lon|x|y', case=False)])
print(df.columns[df.columns.str.contains('lat|lon|x|y|coord', case=False)])


Index(['County', 'LILATracts_1And10', 'LILATracts_halfAnd10',
       'LILATracts_1And20', 'LILATracts_Vehicle', 'PovertyRate',
       'MedianFamilyIncome', 'LATracts_half', 'LATracts1', 'LATracts10',
       'LATracts20', 'LATractsVehicle_20'],
      dtype='object')
Index(['County', 'LILATracts_1And10', 'LILATracts_halfAnd10',
       'LILATracts_1And20', 'LILATracts_Vehicle', 'PovertyRate',
       'MedianFamilyIncome', 'LATracts_half', 'LATracts1', 'LATracts10',
       'LATracts20', 'LATractsVehicle_20'],
      dtype='object')


In [ ]:
!pip install geemap


In [ ]:
# --- Install & Import Packages ---
#!pip install geemap geopandas

import ee
import geemap
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# --- Authenticate Earth Engine ---
from google.oauth2 import service_account
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials)

# --- Load Food Access CSV ---
df = pd.read_csv('/content/Food Access Research Atlas.csv', encoding='latin1')

# Filter for Arizona only (assuming there's a 'State' column)
df_az = df[df['State'] == 'AZ'].copy()

# Ensure Census Tract IDs are in 11-digit format to match TIGER/Line data
df_az['CensusTract'] = df_az['CensusTract'].astype(str).str.zfill(11)

# --- Filter for Low-Income & Low-Access Tracts ---
df_filtered = df_az[(df_az['LowIncomeTracts'] == 1) & (df_az['LILATracts_1And10'] == 1)].copy()

# --- Load TIGER/Line Shapefile for Arizona Census Tracts ---
tracts_az = gpd.read_file("/content/tl_2020_04_tract.zip")

# Merge the Food Access data with the TIGER/Line geometries using the Census tract IDs
merged_gdf = tracts_az.merge(df_filtered, left_on='GEOID', right_on='CensusTract')

# Check the data
print("Merged GeoDataFrame shape:", merged_gdf.shape)
print(merged_gdf.head())

# Reproject to WGS84 for compatibility with basemaps
merged_gdf = merged_gdf.to_crs("EPSG:4326")

# --- Create Interactive Map ---
Map = geemap.Map(center=[34, -111.5], zoom=6)
Map.add_basemap("HYBRID")  # Add an explicit basemap
Map.add_gdf(merged_gdf, layer_name='Low-Income Low-Access Tracts')
Map.addLayerControl()

# Display the map in the notebook
Map

# Save the map to HTMl
Map.save('urban_agriculture_kkkkk.html')


DataSourceError: '/vsizip//content/tl_2020_04_tract.zip' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
!pip install geopandas geemap shapely fiona

import requests
import io
import zipfile
import geopandas as gpd
import geemap

# Download the ZIP in memory
url = "https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_04_tract.zip"
response = requests.get(url)
response.raise_for_status()

# Extract it in memory
z = zipfile.ZipFile(io.BytesIO(response.content))

# Read shapefile from the in-memory ZIP
tracts_az = gpd.read_file(z)
print("Shape:", tracts_az.shape)
print(tracts_az.head())

# Reproject to WGS84 (EPSG:4326)
tracts_az = tracts_az.to_crs(epsg=4326)

# Create a map
Map = geemap.Map(center=[34, -111.5], zoom=6)
Map.add_basemap("HYBRID")
Map.add_gdf(tracts_az, layer_name='Arizona Census Tracts')
Map.addLayerControl()
Map


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 45.2 MB/s eta 0:00:00


TypeError: ZipFile.read() missing 1 required positional argument: 'name'

#Chris

#Shivani

In [ ]:
pip install earthengine-api

In [ ]:
import ee
from google.oauth2 import service_account

# Path to your downloaded JSON key file
SERVICE_ACCOUNT_KEY_FILE = '/content/spacehack2025-454422-764219dc03b0.json'

# Service account email
SERVICE_ACCOUNT_EMAIL = 'spacehack@spacehack2025-454422.iam.gserviceaccount.com'

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)

# Initialize Earth Engine with credentials
ee.Initialize(credentials)


In [ ]:
collections = ee.data.getList({'id': 'LANDSAT'})
for c in collections:
    print(c['id'])



projects/earthengine-public/assets/LANDSAT/COMPOSITES
projects/earthengine-public/assets/LANDSAT/GLS1975
projects/earthengine-public/assets/LANDSAT/GLS1975_MOSAIC
projects/earthengine-public/assets/LANDSAT/GLS1975_TOA_MOSAIC
projects/earthengine-public/assets/LANDSAT/GLS2005
projects/earthengine-public/assets/LANDSAT/GLS2005_L5
projects/earthengine-public/assets/LANDSAT/GLS2005_L7
projects/earthengine-public/assets/LANDSAT/GLS2005_MAPPED
projects/earthengine-public/assets/LANDSAT/L4
projects/earthengine-public/assets/LANDSAT/L4_L1T
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_BAI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_EVI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NBRT
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NDSI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NDVI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_NDWI
projects/earthengine-public/assets/LANDSAT/L4_L1T_32DAY_RAW
projects/earthengine-public/assets/LAN

In [ ]:
# Step 3: Define the Area of Interest (Phoenix, Arizona)
phoenix = ee.Geometry.Point([-112.0740, 33.4484]).buffer(10000)  # 10 km buffer


In [ ]:
# Step 4: Pull NDVI data from Sentinel-2
sentinel2 = ee.ImageCollection("COPERNICUS/S2")
filtered_s2 = sentinel2.filterBounds(phoenix).filterDate('2022-01-01', '2022-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
ndvi = filtered_s2.mean().normalizedDifference(['B8', 'B4']).clip(phoenix)


/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
# Step 5: Visualize the results
import folium

# Define a method for displaying Earth Engine images on a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on Phoenix
my_map = folium.Map(location=[33.4484, -112.0740], zoom_start=12)

# Add the NDVI layer to the map
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}
my_map.add_ee_layer(ndvi, vis_params, 'NDVI')

# Display the map
my_map.save('ndvi_map.html')

## Identifying areas in Phoenix

### Phoenix Area

In [ ]:
# Define Phoenix, Arizona, as a point and create a buffer around it
phoenix = ee.Geometry.Point([-112.0740, 33.4484]).buffer(10000)  # 10 km buffer

### Pull Relevant Satellite Data


In [ ]:
# Land Use/ Land Cover Data

lulc = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD")
lulc_2019 = lulc.filter(ee.Filter.eq('system:index', '2019')).first()
phoenix_lulc = lulc_2019.clip(phoenix)

In [ ]:
# Vegetation Indices (NDVI)

sentinel2 = ee.ImageCollection("COPERNICUS/S2")
filtered_s2 = sentinel2.filterBounds(phoenix).filterDate('2022-01-01', '2022-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
ndvi = filtered_s2.mean().normalizedDifference(['B8', 'B4']).clip(phoenix)

In [ ]:
# Soil Moisture Data

smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")
smap_filtered = smap.filterBounds(phoenix).filterDate('2022-01-01', '2022-12-31')
soil_moisture = smap_filtered.mean().clip(phoenix)

In [ ]:
# Urban Infrastructure Data

osm = ee.FeatureCollection("OpenStreetMap/OpenStreetMap")
urban_areas = osm.filter(ee.Filter.eq('landuse', 'residential')).filterBounds(phoenix)

## Analyze the Data

In [ ]:
# Extract specific land cover classes (e.g., barren land, open space)
barren_land = phoenix_lulc.eq(31)  # Class 31 represents barren land

In [ ]:
low_vegetation = ndvi.lt(0.2)  # Threshold for low vegetation

In [ ]:
suitable_soil = soil_moisture.gt(0.2)  # Threshold for suitable soil moisture

In [ ]:
suitable_sites = barren_land.And(low_vegetation).And(suitable_soil)

In [ ]:
# Ensure barren_land is a single-band image
#barren_land = phoenix_lulc.eq(31)  # Class 31 represents barren land
barren_land = phoenix_lulc.select('landcover').eq(31)  # Select the 'landcover' band

# Ensure low_vegetation is a single-band image
#low_vegetation = ndvi.select('NDVI').lt(0.2)  # Select the 'NDVI' band and apply threshold
low_vegetation = ndvi.select('nd').lt(0.2)  # Use the correct band name 'nd'


# Ensure suitable_soil is a single-band image
# suitable_soil = soil_moisture.select('moisture_band').gt(0.2)  # Select the moisture band and apply threshold
suitable_soil = soil_moisture.select('sm_surface').gt(0.2)

# Combine the conditions using logical AND
suitable_sites = barren_land.And(low_vegetation).And(suitable_soil)

In [ ]:
print("Soil moisture bands:", soil_moisture.bandNames().getInfo())


Soil moisture bands: ['sm_surface', 'sm_rootzone', 'sm_profile', 'sm_surface_wetness', 'sm_rootzone_wetness', 'sm_profile_wetness', 'surface_temp', 'soil_temp_layer1', 'soil_temp_layer2', 'soil_temp_layer3', 'soil_temp_layer4', 'soil_temp_layer5', 'soil_temp_layer6', 'snow_mass', 'snow_depth', 'land_evapotranspiration_flux', 'overland_runoff_flux', 'baseflow_flux', 'snow_melt_flux', 'soil_water_infiltration_flux', 'land_fraction_saturated', 'land_fraction_unsaturated', 'land_fraction_wilting', 'land_fraction_snow_covered', 'heat_flux_sensible', 'heat_flux_latent', 'heat_flux_ground', 'net_downward_shortwave_flux', 'net_downward_longwave_flux', 'radiation_shortwave_downward_flux', 'radiation_longwave_absorbed_flux', 'precipitation_total_surface_flux', 'snowfall_surface_flux', 'surface_pressure', 'height_lowatmmodlay', 'temp_lowatmmodlay', 'specific_humidity_lowatmmodlay', 'windspeed_lowatmmodlay', 'vegetation_greenness_fraction', 'leaf_area_index', 'sm_rootzone_pctl', 'sm_profile_pctl',

In [ ]:
print("Barren land bands:", barren_land.bandNames().getInfo())
print("Low vegetation bands:", low_vegetation.bandNames().getInfo())
print("Suitable soil bands:", suitable_soil.bandNames().getInfo())

Barren land bands: ['landcover']
Low vegetation bands: ['nd']
Suitable soil bands: ['sm_surface']


## Visualize the Data

In [ ]:
print(suitable_sites.bandNames().getInfo())

['landcover']


In [ ]:
import folium

# Define a method for displaying Earth Engine images on a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on Phoenix
my_map = folium.Map(location=[33.4484, -112.0740], zoom_start=12)

# Add the suitable sites layer to the map
vis_params = {'min': 0, 'max': 1, 'palette': ['red', 'green']}
my_map.add_ee_layer(suitable_sites, vis_params, 'Suitable Sites')

# Display the map
my_map.save('urban_agriculture_map.html')

Test Code

In [ ]:

# Define Phoenix, Arizona, as a point and create a buffer around it
phoenix_point = ee.Geometry.Point([-112.0740, 33.4484])
phoenix_buffer = phoenix_point.buffer(10000)  # 10 km buffer

# Land Use/Land Cover Data
lulc = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").sort('system:time_start', False).first()
lulc = lulc.select('landcover').clip(phoenix_buffer)

# Vegetation Indices (NDVI)
sentinel2 = ee.ImageCollection("COPERNICUS/S2")
filtered_s2 = sentinel2.filterBounds(phoenix_buffer).filterDate('2019-01-01', '2019-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
ndvi = filtered_s2.mean().normalizedDifference(['B8', 'B4']).rename('ndvi').clip(phoenix_buffer)

# Soil Moisture Data
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")
smap_filtered = smap.filterBounds(phoenix_buffer).filterDate('2019-01-01', '2019-12-31')
soil_moisture = smap_filtered.mean().select('sm_surface').clip(phoenix_buffer)

# Ensure barren_land is correctly classified
barren_land = lulc.eq(31)  # Class 31 represents barren land

# Ensure low_vegetation is a single-band image
low_vegetation = ndvi.lt(0.2)

# Ensure suitable_soil is a single-band image
suitable_soil = soil_moisture.gt(0.2)

# Combine the conditions using multiplication
suitable_sites = barren_land.multiply(low_vegetation).multiply(suitable_soil)

# Define a method for displaying Earth Engine images on a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on Phoenix
my_map = folium.Map(location=[33.4484, -112.0740], zoom_start=12)

# Add the suitable sites layer to the map with a gradient color palette
vis_params = {'min': 0, 'max': 1, 'palette': ['blue', 'yellow', 'green']}  # Gradient from blue (unsuitable) to green (suitable)
my_map.add_ee_layer(suitable_sites, vis_params, 'Suitable Sites')

# Add the 10 km buffer perimeter to the map
folium.GeoJson(
    phoenix_buffer.getInfo(),
    name='10 km Buffer',
    style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
).add_to(my_map)

# Add layer control to toggle layers
folium.LayerControl().add_to(my_map)

# Save the map to an HTML file
my_map.save('urban_agriculture_map.html')


#Amoreena

In [ ]:
type(suitable_sites)
suitable_sites.getInfo()

{'type': 'Image',
 'bands': [{'id': 'landcover',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'dimensions': [664, 667],
   'origin': [33528, 67381],
   'crs': 'PROJCS["IMAGINE GeoTIFF Support\nERDAS IMAGINE 2018  16.5.0.596\nProjection = Albers Conical Equal Area", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Albers_Conic_Equal_Area"], \n  PARAMETER["central_meridian", -96.0], \n  PARAMETER["latitude_of_origin", 23.0], \n  PARAMETER["standard_parallel_1", 29.5], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["standard_parallel_2", 45.5], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]',
   'crs_tr

#Kadn

In [ ]:
import ee
from google.oauth2 import service_account

# Path to your downloaded JSON key file
SERVICE_ACCOUNT_KEY_FILE = '/spacehack2025-454422-764219dc03b0.json'

# Service account email
SERVICE_ACCOUNT_EMAIL = 'spacehack@spacehack2025-454422.iam.gserviceaccount.com'

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine']
)

# Initialize Earth Engine with credentials
ee.Initialize(credentials)


Kadn Testing

In [ ]:
import ee
import folium
from folium import TileLayer

# Load Arizona counties (from US Census TIGER data)
az_counties = ee.FeatureCollection("TIGER/2018/Counties") \
                 .filter(ee.Filter.eq('STATEFP', '04'))  # Arizona = 04

# Merge all counties into one region
az_geom = az_counties.geometry()

# Load Cropland (GFSAD1000) and apply transparency
cropland = ee.Image('USGS/GFSAD1000_V1').clip(az_geom).selfMask()

# Load WorldPop population density (2020)
population = ee.ImageCollection("WorldPop/GP/100m/pop") \
                .filterDate('2020-01-01', '2020-12-31') \
                .first() \
                .clip(az_geom) \
                .selfMask()

# Identify "Surplus Zones" = Cropland + Low Population (<20 people per pixel)
cropland_mask = cropland.eq(1)
low_pop_mask = population.lt(20)
surplus_zone = cropland_mask.And(low_pop_mask).selfMask()

# Visualization settings
cropland_vis = {'min': 0, 'max': 1, 'palette': ['green'], 'opacity': 0.4}  # Semi-transparent
population_vis = {'min': 0, 'max': 300, 'palette': ['blue', 'red'], 'opacity': 0.5}  # Gradient
surplus_vis = {'min': 0, 'max': 1, 'palette': ['yellow'], 'opacity': 0.7}  # Most visible

# Create an interactive map centered in Phoenix, AZ
m = folium.Map(location=[33.45, -112.07], zoom_start=7)

# Add Cropland (Green) - Background layer
TileLayer(
    tiles=cropland.getMapId(cropland_vis)['tile_fetcher'].url_format,
    attr='GFSAD Cropland',
    name='Cropland (Farmland)',
    overlay=True,
    control=True
).add_to(m)

# Add Population (Blue-Red) - Mid-layer
TileLayer(
    tiles=population.getMapId(population_vis)['tile_fetcher'].url_format,
    attr='WorldPop Population',
    name='Population Density',
    overlay=True,
    control=True
).add_to(m)

# Add Surplus Zones (Yellow) - Top-most layer
TileLayer(
    tiles=surplus_zone.getMapId(surplus_vis)['tile_fetcher'].url_format,
    attr='Surplus Zones',
    name='Likely Surplus Areas',
    overlay=True,
    control=True
).add_to(m)

# Add layer controls
folium.LayerControl().add_to(m)

# Display the map
m


In [ ]:
import ee
import folium
from folium import TileLayer

# Define Phoenix, Arizona, as a point and create a buffer around it
phoenix_point = ee.Geometry.Point([-112.0740, 33.4484])
phoenix_buffer = phoenix_point.buffer(30000)  # 13 km buffer

# Land Use/Land Cover Data
lulc = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").sort('system:time_start', False).first()
lulc = lulc.select('landcover').clip(phoenix_buffer)

# Vegetation Indices (NDVI)
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR")
filtered_s2 = sentinel2.filterBounds(phoenix_buffer).filterDate('2019-01-01', '2019-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
ndvi = filtered_s2.mean().normalizedDifference(['B8', 'B4']).rename('ndvi').clip(phoenix_buffer)

# Soil Moisture Data
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")
smap_filtered = smap.filterBounds(phoenix_buffer).filterDate('2019-01-01', '2019-12-31')
soil_moisture = smap_filtered.mean().select('sm_surface').clip(phoenix_buffer)


# Ensure barren_land is correctly classified
#barren_or_open = lulc.eq(31).Or(lulc.eq(21)).Or(lulc.eq(12)).Or(lulc.eq(22)).Or(lulc.eq(71)).Or(lulc.eq(81)).Or(lulc.eq(52)).Or(lulc.eq(23)).Or(lulc.eq(24))
urban_classes = ee.List([31, 71, 21])
urban_mask = lulc.remap(urban_classes, ee.List.repeat(1, urban_classes.length())).selfMask()


# Ensure low_vegetation is a single-band image
#low_vegetation = ndvi.lt(0.2)

# Ensure suitable_soil is a single-band image
#suitable_soil = soil_moisture.gt(0.2)



# Combine with logical OR instead of strict AND
#suitable_sites = (barren_land.add(low_vegetation).add(suitable_soil)).gt(1).selfMask()


# Define a method for displaying Earth Engine images on a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on Phoenix
my_map = folium.Map(location=[33.4484, -112.0740], zoom_start=12)

# Add the suitable sites layer to the map with a gradient color palette
vis_params = {'min': 0, 'max': 1, 'palette': ['blue', 'yellow', 'green']}  # Gradient from blue (unsuitable) to green (suitable)

#my_map.add_ee_layer(low_vegetation.selfMask(), {'palette': ['brown'], 'opacity': 0.6}, 'Low Vegetation')

#my_map.add_ee_layer(suitable_soil.selfMask(), {'palette': ['black']}, 'Suitable Soil')
suitable_soil = soil_moisture.gt(0.02).selfMask()

my_map.add_ee_layer(
    suitable_soil,
    {'palette': ['black'], 'opacity': 0.6},
    'Urban Areas (21–24)'
)




my_map.add_ee_layer(
    urban_mask,
    {'palette': ['blue'], 'opacity': 0.6},
    'Urban Areas (21–24)'
)


# Add the 10 km buffer perimeter to the map
folium.GeoJson(
    phoenix_buffer.getInfo(),
    name='10 km Buffer',
    style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
).add_to(my_map)

# Add layer control to toggle layers
folium.LayerControl().add_to(my_map)

# Save the map to an HTML file
my_map.save('urban_agriculture_map2.html')


# usda




In [ ]:
# Install necessary libraries
!pip install geemap geopandas rasterio folium plotly

# Import libraries
import ee
import geemap
import geopandas as gpd
import rasterio
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

print("Libraries imported and GEE initialized successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 44.7 MB/s eta 0:00:00
Libraries imported and GEE initialized successfully!


In [ ]:
# Create an interactive map using geemap
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Install necessary libraries
!pip install geemap geopandas rasterio folium plotly


# Import libraries
import ee
import geemap
import geopandas as gpd
import rasterio
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


print("Libraries imported and GEE initialized successfully!")
# Create an interactive map using geemap
Map = geemap.Map()
Map
# Load USDA NASS Cropland Data Layer for 2022
cdl = ee.ImageCollection('USDA/NASS/CDL')
cdl_2022 = cdl.filterDate('2022-01-01', '2022-12-31').first()


# Add the layer to the map
Map.addLayer(cdl_2022, {}, 'Cropland Data Layer 2022')
Map
# Load MODIS NDVI dataset
modis_ndvi = ee.ImageCollection('MODIS/006/MOD13Q1')


# Filter for the most recent year
recent_ndvi = modis_ndvi.filterDate('2022-01-01', '2022-12-31').first()


# Select the NDVI band
ndvi = recent_ndvi.select('NDVI')


# Add the NDVI layer to the map
Map.addLayer(ndvi, {'min': 0, 'max': 9000, 'palette': ['red', 'yellow', 'green']}, 'NDVI 2022')
Map
# Load Landsat 8 imagery
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')


# Filter for the Phoenix metro area and a specific time range
phoenix_roi = ee.Geometry.Rectangle([-112.5, 33.0, -111.5, 34.0])  # Approximate Phoenix metro area
landsat_phoenix = landsat.filterBounds(phoenix_roi).filterDate('2015-01-01', '2023-01-01')


# Select the first and last images for comparison
first_image = landsat_phoenix.first()
last_image = landsat_phoenix.sort('system:time_start', False).first()


# Add the images to the map
Map.addLayer(first_image, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 3000}, 'Landsat 2015')
Map.addLayer(last_image, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 3000}, 'Landsat 2023')
Map
# Load USDA Food Access Atlas data (replace with your file path)
food_access = gpd.read_file('/content/Food Access Research Atlas.csv')


# Display the first few rows
print(food_access.head())
# Example: Overlay food access data on the map
food_access_phoenix = food_access[food_access.geometry.intersects(phoenix_roi)]


# Add food access data to the map
Map.addLayer(food_access_phoenix, {'color': 'red'}, 'Food Access Data')
Map



Libraries imported and GEE initialized successfully!
  CensusTract    State          County Urban Pop2010 OHU2010  \
0  1001020100  Alabama  Autauga County     1    1912     693   
1  1001020200  Alabama  Autauga County     1    2170     743   
2  1001020300  Alabama  Autauga County     1    3373    1256   
3  1001020400  Alabama  Autauga County     1    4386    1722   
4  1001020500  Alabama  Autauga County     1   10766    4082   

  GroupQuartersFlag NUMGQTRS PCTGQTRS LILATracts_1And10  ... TractSeniors  \
0                 0        0        0                 0  ...          221   
1                 0      181     8.34                 1  ...          214   
2                 0        0        0                 0  ...          439   
3                 0        0        0                 0  ...          904   
4                 0      181     1.68                 0  ...         1126   

  TractWhite TractBlack TractAsian TractNHOPI TractAIAN TractOMultir  \
0       1622        217    

AttributeError: 'DataFrame' object has no attribute 'geometry'